In [1]:
import selenium
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

In [2]:
driver = webdriver.Chrome(r"D://chromedriver.exe")

In [3]:
driver.maximize_window()

In [4]:
products = ['laptops','mobiles','headphones','smart watches','printers','monitors','routers','speakers','home theatres','storage devices']

In [5]:
# Fetching the website
driver.get("https://www.amazon.in")

In [24]:
def get_results(driver, prod_name):
    # Entering input item in  search field
    item_inp = driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']")
    item_inp.send_keys(prod_name)
    print("Value Entered in Search Box...")
    time.sleep(1)

    # Clicking Search Button
    search_btn =  driver.find_element_by_xpath("//span[@id='nav-search-submit-text']")
    search_btn.click()
    print("Search Button Clicked...")
    time.sleep(2)

    # Sorting by customer reviews
    bttn = driver.find_element_by_xpath("//span[@class='a-button-text a-declarative']")
    bttn.click()
    time.sleep(2)
    opn = driver.find_element_by_xpath("//a[contains(text(),'Avg. Customer Review')]")
    opn.click()
    print("Sorting by Customer reviews...")
    time.sleep(2)

In [25]:
def fetch_url(driver):
    #Fetching first 100 Product URLs
    url = []

    while len(url)<=100:
        try:
            temp = driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]')
            for i in temp:
                url.append(i.get_attribute('href'))
            try:
                next_btn = driver.find_element_by_xpath("//li[@class='a-last']")
                next_btn.click()
                time.sleep(2)
            except NoSuchElementException:
                print("Displayed Results less than three pages")
        except:
            print("Exception")
    print("------------------------------------------URLS FETCHED------------------------------------------")
    print("TOTAL URLS : ", len(url))
    return url

In [26]:
def get_reviews(prod_name, driver, url):
    reviews = pd.DataFrame(columns = ['Product_type', 'Rating', 'Review_title', 'Review_Date', 'Review'])
    count = 1
    for i in url:
        driver.get(i)
        time.sleep(2)

        try:
            rev = driver.find_element_by_xpath("//a[contains(text(),'See all reviews')]")
            rev.click()

        except NoSuchElementException as e:
            print("Exception")
        
        except Exception as e:
            print(e)
            return reviews

        try:
            trans = driver.find_element_by_xpath("//a[contains(text(),'Translate all reviews to English')]")
            trans.click()

        except:
            print("Translation Exception")
        print('==========================================PRODUCT',count,'==========================================')
        temp_reviews = prod_review_fetch(prod_name, driver)
        temp_reviews_df = pd.DataFrame(data = temp_reviews, columns = ['Product_type', 'Rating', 'Review_title', 'Review_Date', 'Review'])
        reviews = pd.concat([reviews, temp_reviews_df])
        count+=1
    return reviews

In [27]:
def star_change(j,driver):
    if(j!=0):
        try:
            star_select = driver.find_element_by_xpath("//div[@class='a-column a-span3 star-rating-select']")
            star_select.click()
            time.sleep(1)
        except:
            print("Star Selection Exception")

        try:
            x_star = driver.find_element_by_xpath("//a[contains(text(),'"+str(j)+" star only')]")
            x_star.click()
            time.sleep(2)
        except:
            print(j,"Star exception")
    else:
        return 'break'

In [28]:
def prod_review_fetch(prod_name, driver):
    reviews = []
    j=4
    k=1
    try:
        star_select = driver.find_element_by_xpath("//div[@class='a-column a-span3 star-rating-select']")
        star_select.click()
        time.sleep(1)
    except:
        print("Star Selection Exception")

    try:
        x_star = driver.find_element_by_xpath("//a[contains(text(), '5 star only')]")
        x_star.click()
        time.sleep(2)
    except:
        print("5 Star exception")


    while True:

        try:
            review_list = driver.find_elements_by_xpath("//div[@data-hook='review']")
        except NoSuchElementException as e:
            print("Exception No reviews for Product")
        except Exception as e:
            break

        for i in range(len(review_list)):
            path = "//div[@data-hook='review']["+str(i+1)+"]//"

            try:
                rev_title = driver.find_element_by_xpath(path+"span[@data-hook='review-title']").text
            except Exception as e:
                try:
                    rev_title = driver.find_element_by_xpath(path+"a[@data-hook='review-title']").text
                except Exception as e:
                    print('Rev_title_Exception')
                    print(e)
                    continue

            try:
                revs = driver.find_element_by_xpath(path+"div[@class='a-row a-spacing-small review-data']").text
            except Exception as e:
                print('Rev_Exception')
                print(e)
                continue

            try:
                date_place = driver.find_element_by_xpath(path+"span[@data-hook='review-date']").text
                dp_list = date_place.split('on')
                rev_date = dp_list[1].strip()
            except Exception as e:
                print('Rev_Date_Exception')
                print(e)
                continue

            temp = [prod_name, j+1, rev_title, rev_date, revs]
            reviews.append(temp)

        try:
            if(len(reviews)>=10*k):
                if star_change(j,driver)!='break':
                    print("End Of",j+1,"star reviews")
                    j=j-1
                    k=k+1
                else:
                    print("End Of",1,"star reviews")
                    print("------End of Reviews for Product-----")
                    break
            else:
                next_btn = driver.find_element_by_xpath("//li[@class='a-last']")
                next_btn.click()
                time.sleep(2)
                print("Next Page   : ",j+1,"stars")
        except NoSuchElementException:
            if star_change(j,driver)!='break':
                j=j-1
                k=k+1
                print("End Of",j+1,"star reviews")

            else:
                print("End Of",1,"star reviews")
                print("------End of Reviews for Product-----")
                break
        except Exception as e:
            break
                
    return reviews

In [29]:
df = pd.DataFrame(columns = ['Product_type', 'Rating', 'Review_title', 'Review_Date', 'Review'])
df

Empty DataFrame
Columns: [Product_type, Rating, Review_title, Review_Date, Review]
Index: []

In [30]:
for i in products[5:]:
    print('\033[1m','******************************************',i.upper(),'******************************************','\033[0m')
    get_results(driver,i)
    links = fetch_url(driver)
    prod_reviews = get_reviews(i,driver,links)
    df = pd.concat([df, prod_reviews])

 ****************************************** MONITORS ****************************************** 
Value Entered in Search Box...
Search Button Clicked...
Sorting by Customer reviews...
------------------------------------------URLS FETCHED------------------------------------------
TOTAL URLS :  115
Translation Exception
==========================================PRODUCT 1 ==========================================
End Of 5 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 2 ==========================================
End Of 5 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 3 ==========================================
End Of 5 star reviews
End Of 3 star reviews
End Of 2 s

Translation Exception
==========================================PRODUCT 30 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
End Of 1 star reviews
Next Page   :  1 stars
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 31 ==========================================
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 32 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 33 ==========================================
End Of 5 star reviews
3 Star exception
End Of 4 star reviews


End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 60 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 61 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 62 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
1 Star exception
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 63 ==========================================
End Of 

End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 90 ==========================================
End Of 5 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
Next Page   :  1 stars
Next Page   :  1 stars
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 91 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 2 star reviews
End Of 1 star reviews
Next Page   :  1 stars
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 92 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
=======

Translation Exception
==========================================PRODUCT 3 ==========================================
End Of 5 star reviews
3 Star exception
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 4 ==========================================
End Of 5 star reviews
3 Star exception
End Of 4 star reviews
End Of 3 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 5 ==========================================
End Of 5 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 6 ==========================================
End Of 5 star reviews
3 Star exception
End Of 4 star r

...

Next Page   :  1 stars
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 43 ==========================================
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 44 ==========================================
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Exception
Translation Exception
==========================================PRODUCT 45 ==========================================
Star Selection Exception
5 Star exception
Star Selection Exception
4 Star exception
End Of 4 star reviews
Star Selection Exception
3 Star exception
End Of 3 star reviews
Star Selection Exception
2 Star exception
End Of 2 star reviews
Star Selection Exceptio

Exception
Translation Exception
==========================================PRODUCT 66 ==========================================
Star Selection Exception
5 Star exception
Star Selection Exception
4 Star exception
End Of 4 star reviews
Star Selection Exception
3 Star exception
End Of 3 star reviews
Star Selection Exception
2 Star exception
End Of 2 star reviews
Star Selection Exception
1 Star exception
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 67 ==========================================
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 68 ==========================================
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews

Exception
Translation Exception
==========================================PRODUCT 85 ==========================================
Star Selection Exception
5 Star exception
Star Selection Exception
4 Star exception
End Of 4 star reviews
Star Selection Exception
3 Star exception
End Of 3 star reviews
Star Selection Exception
2 Star exception
End Of 2 star reviews
Star Selection Exception
1 Star exception
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Exception
Translation Exception
==========================================PRODUCT 86 ==========================================
Star Selection Exception
5 Star exception
Star Selection Exception
4 Star exception
End Of 4 star reviews
Star Selection Exception
3 Star exception
End Of 3 star reviews
Star Selection Exception
2 Star exception
End Of 2 star reviews
Star Selection Exception
1 Star exception
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Exception
Translation Except

Exception
Translation Exception
==========================================PRODUCT 105 ==========================================
Star Selection Exception
5 Star exception
Star Selection Exception
4 Star exception
End Of 4 star reviews
Star Selection Exception
3 Star exception
End Of 3 star reviews
Star Selection Exception
2 Star exception
End Of 2 star reviews
Star Selection Exception
1 Star exception
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Exception
Translation Exception
==========================================PRODUCT 106 ==========================================
Star Selection Exception
5 Star exception
Star Selection Exception
4 Star exception
End Of 4 star reviews
Star Selection Exception
3 Star exception
End Of 3 star reviews
Star Selection Exception
2 Star exception
End Of 2 star reviews
Star Selection Exception
1 Star exception
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Exception
Translation Exce

End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 123 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 124 ==========================================
End Of 4 star reviews
End Of 3 star reviews
End Of 2 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 125 ==========================================
End Of 5 star reviews
End Of 4 star reviews
End Of 3 star reviews
End Of 1 star reviews
End Of 1 star reviews
------End of Reviews for Product-----
Translation Exception
==========================================PRODUCT 1

In [33]:
df.reset_index(inplace=True,drop=True)
df

Product_type  Rating  \
0              laptops       5   
1              laptops       5   
2              laptops       5   
3              laptops       5   
4              laptops       5   
...                ...     ...   
35328  storage devices       1   
35329  storage devices       1   
35330  storage devices       1   
35331  storage devices       1   
35332  storage devices       1   

                                            Review_title        Review_Date  \
0                                     Best Gaming Laptop   8 September 2020   
1      An incredibly capabale gaming laptop that has ...       28 June 2020   
2      Headphones/Headset Static Noise Issue | Sound ...  27 September 2020   
3                              Best Gaming Computer 2020       29 June 2020   
4                                      Not what it looks      19 April 2021   
...                                                  ...                ...   
35328                                          dead slow     7 January 2021   
35329                     This is not an Type C product.    23 January 2020   
35330                                 Speed is the issue        17 May 2021   
35331                                      worst product     9 January 2021   
35332                                        Very Costly    30 October 2020   

                                                  Review  
0      Best gaming laptop for AAA Games and for Creat...  
1      This is a very good laptop that is very capabl...  
2      Hi everyone,\n\nI just want to give a word of ...  
3      I bought this laptop as a last second mobile C...  
4      There is no doubt this is a powerful laptop, b...  
...                                                  ...  
35328  Dont like becoz its dead slow and hard to writ...  
35329  They clearly mentioned as the product is Type ...  
35330  Speed is very slow while copying in or from dr...  
35331  worst harddrive ever.. I got the product from ...  
35332  Very Costly\nAdata 1 tb rs 4099\nWD 1.5 tb rs ...  

[35333 rows x 5 columns]

In [34]:
df['Rating'].value_counts()

5    9125
1    7591
4    7342
3    6026
2    5249
Name: Rating, dtype: int64

In [35]:
df['Product_type'].value_counts()

monitors           4456
headphones         4455
storage devices    4306
smart watches      4216
routers            4162
printers           3456
laptops            3178
speakers           3127
home theatres      2904
mobiles            1073
Name: Product_type, dtype: int64

In [36]:
df.to_csv('Amazon_Reviews.csv', index = False)

In [37]:
driver.close()